In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('nice.mplstyle')

In [ ]:
def M(Ng):
    return 0.52 * (Ng/500)**3 + 0.58

In [ ]:
fig, ax = plt.subplots()
ncore = np.linspace(1, 5000, 100)
